In [1]:
%pip install mediapipe==0.10.11
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
import random
import gc
import glob
import time
import json
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm
from itertools import product, chain
from collections import defaultdict

In [88]:
# *: 파일 경로가 ...\asdf\asdf.(확장자) 형태일시 추가
video_root_N = r'D:\project\Data\Data\Data\Training\SourceData\TS\Video\N\N\*'
video_root_BY = r'D:\project\Data\Data\Data\Training\SourceData\TS\Video\Y\BY\*'
json_root = r'D:\landmark_train_N_json'
json_root_BY = r'D:\project\New_Data\my_json_BY'

RES_OF_MONITER = (1920, 1080)
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]
LINE_LIST = [[11, 12], [11, 15], [11, 23], [12, 16], [12, 24], [23, 24], [23, 25], [25, 27], [24, 26], [26, 28]]
COLOR_PINK = (153, 51, 255)
COLOR_WHITE = (255, 255, 255)
FRAME_INTERVAL = 5

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)

In [89]:
def draw_line_plot(pose_xyz_dic, elev, azim, roll, w, h, z_ratio, extent):

    fontlabel = {"fontsize":"large", "color":"gray", "fontweight":"bold"}
    fig, ax = plt.subplots(ncols=1, figsize=(5, 5), subplot_kw={"projection":"3d"})
    ax.view_init(elev=elev, azim = azim, roll=roll)
    ax.set_xlabel('X', fontdict=fontlabel, labelpad=8)
    ax.set_ylabel('Y', fontdict=fontlabel, labelpad=8)
    ax.set_zlabel('Z', fontdict=fontlabel, labelpad=8)
    ax.set_xlim(int(extent['x']['m']*w), int(extent['x']['M']*w))
    ax.set_ylim(int(extent['y']['m']*h), int(extent['y']['M']*h))
    ax.set_zlim(extent['z']['m'], extent['z']['M'])
    ax.set_box_aspect([
        extent['x']['M'] - extent['x']['m'], 
        extent['y']['M'] - extent['y']['m'], 
        z_ratio
        ])
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_zlim(ax.get_zlim()[::-1])

    for line in LINE_LIST:
        lx1, ly1, lz1 = pose_xyz_dic[f'landmark_{line[0]}']['x'], pose_xyz_dic[f'landmark_{line[0]}']['y'], pose_xyz_dic[f'landmark_{line[0]}']['z']
        lx2, ly2, lz2 = pose_xyz_dic[f'landmark_{line[1]}']['x'], pose_xyz_dic[f'landmark_{line[1]}']['y'], pose_xyz_dic[f'landmark_{line[1]}']['z']
        lx1, lx2 = lx1 * w, lx2 * w
        ly1, ly2 = ly1 * h, ly2 * h
        ax.plot([lx1, lx2], [ly1, ly2], [lz1, lz2])

    for idx in DESIRED_LANDMARKS:
        ax.scatter(pose_xyz_dic[f'landmark_{idx}']['x']*w, pose_xyz_dic[f'landmark_{idx}']['y']*h, pose_xyz_dic[f'landmark_{idx}']['z'])
        ax.text(pose_xyz_dic[f'landmark_{idx}']['x']*w, pose_xyz_dic[f'landmark_{idx}']['y']*h, pose_xyz_dic[f'landmark_{idx}']['z'], idx)

    fig.canvas.draw()
    img_plot = np.array(fig.canvas.renderer.buffer_rgba())
    plt.close(fig)
    
    return img_plot
    

In [90]:
def draw_line_plot1(pose_xyz_dic, elev, azim, roll, w, h, z_ratio, extent):

    fontlabel = {"fontsize":"large", "color":"gray", "fontweight":"bold"}
    fig, ax = plt.subplots(ncols=1, figsize=(5, 5), subplot_kw={"projection":"3d"})
    ax.view_init(elev=elev, azim = azim, roll=roll)
    ax.set_xlabel('Z', fontdict=fontlabel, labelpad=8)
    ax.set_ylabel('X', fontdict=fontlabel, labelpad=8)
    ax.set_zlabel('Y', fontdict=fontlabel, labelpad=8)
    ax.set_ylim(int(extent['x']['m']*w), int(extent['x']['M']*w))
    ax.set_zlim(int(extent['y']['m']*h), int(extent['y']['M']*h))
    ax.set_xlim(extent['z']['m'], extent['z']['M'])
    ax.set_box_aspect([
        z_ratio,
        extent['x']['M'] - extent['x']['m'], 
        extent['y']['M'] - extent['y']['m'],
        ])
    ax.set_zlim(ax.get_zlim()[::-1])
    ax.set_xlim(ax.get_xlim()[::-1])

    for line in LINE_LIST:
        lx1, ly1, lz1 = pose_xyz_dic[f'landmark_{line[0]}']['x'], pose_xyz_dic[f'landmark_{line[0]}']['y'], pose_xyz_dic[f'landmark_{line[0]}']['z']
        lx2, ly2, lz2 = pose_xyz_dic[f'landmark_{line[1]}']['x'], pose_xyz_dic[f'landmark_{line[1]}']['y'], pose_xyz_dic[f'landmark_{line[1]}']['z']
        lx1, lx2 = lx1 * w, lx2 * w
        ly1, ly2 = ly1 * h, ly2 * h
        ax.plot([lz1, lz2], [lx1, lx2], [ly1, ly2])

    for idx in DESIRED_LANDMARKS:
        ax.scatter(pose_xyz_dic[f'landmark_{idx}']['z'], pose_xyz_dic[f'landmark_{idx}']['x']*w, pose_xyz_dic[f'landmark_{idx}']['y']*h)
        ax.text(pose_xyz_dic[f'landmark_{idx}']['z'], pose_xyz_dic[f'landmark_{idx}']['x']*w, pose_xyz_dic[f'landmark_{idx}']['y']*h, idx)

    fig.canvas.draw()
    img_plot = np.array(fig.canvas.renderer.buffer_rgba())
    plt.close(fig)
    
    return img_plot
    

In [105]:
def pose_3D(video_root, json_root):
    json_list = glob.glob(f'{json_root}\\*')
    for json_path in json_list:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            vid_name = json_path.split('\\')[-1].replace('.json', '')
            video_path = video_root.replace('*', vid_name) + '\\' +  vid_name + '.mp4'
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print('Not opened!')
                continue
            
            pose_data = defaultdict(list)
            frame_count = 0
            frame_null = 0
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            new_h = RES_OF_MONITER[1] // 2
            new_w = int(new_h / h * w)
            angle_elev = 0
            angle_azim = -10
            angle_roll = 0
            extent = {'x': {'m': 9999, 'M': -9999}, 'y': {'m': 9999, 'M': 0}, 'z': {'m': 9999, 'M': -9999}}
            # 최대/최소 x, y, z 값 구하기 & 빈 프레임 채우기
            for i, j, k in product(('x', 'y', 'z'), [n for n in range(0, total_frames, FRAME_INTERVAL)], DESIRED_LANDMARKS):
                if f'frame_{j}' not in data['pose_data']:
                    temp_fn = j
                    while f'frame_{frame_null}' not in data['pose_data']:
                        frame_null += 5
                    data['pose_data'][f'frame_{j}'] = data['pose_data'][f'frame_{frame_null}']
                temp = data['pose_data'][f'frame_{j}'][f'landmark_{k}'][i]
                extent[i]['m']= min(extent[i]['m'], temp)
                extent[i]['M']= max(extent[i]['M'], temp)

            z_ratio = (extent['y']['M'] - extent['y']['m']) / 2

            vid_next = False
            while cap.isOpened():
                success, frame = cap.read()
                if not success:
                    break

                # 프레임 건너뛰기 (예: 매 5번째 프레임만 처리)
                if frame_count % 5 != 0:
                    frame_count += 1
                    continue
                
                frame = cv2.resize(frame, (new_w, new_h))

                # if f'frame_{frame_count}' not in data['pose_data']:
                #     print('!')
                #     frame_null = frame_count
                #     while f'frame_{frame_null}' not in data['pose_data']:
                #         frame_null += 5
                #     data['pose_data'][f'frame_{frame_count}'] = data['pose_data'][f'frame_{frame_null}']
                pose_xyz_dic = data['pose_data'][f'frame_{frame_count}']

                img_plot = draw_line_plot1(pose_xyz_dic, angle_elev, angle_azim, angle_roll, new_w, new_h, z_ratio, extent)

                cv2.putText(frame, str(frame_count), (10, 60), cv2.FONT_HERSHEY_DUPLEX, 2, COLOR_PINK, 1, lineType=cv2.LINE_AA)
                img_plot = cv2.putText(img_plot, f'azim: {angle_azim}, elev: {angle_elev}, roll: {angle_roll}', (10, 60), cv2.FONT_HERSHEY_DUPLEX, 1, COLOR_PINK, 1, lineType=cv2.LINE_AA)
                cv2.imshow('image. press q to quit, p to pause, n to pass', frame)
                cv2.imshow('plot', img_plot)
                if frame_count == 0: cv2.waitKey(20)
                key = cv2.waitKey(5) & 0xFF
                if key == ord('q'):
                    cv2.destroyAllWindows()
                    cap.release()
                    return 0
                if key == ord('n'):
                    vid_next = True
                    cv2.destroyAllWindows()
                    cap.release()
                    break
                if key == ord('p'):
                    while True:
                        frame = cv2.putText(frame, 'Paused. Use ASD & ZXC to change the angle of plot.', (10, 90), cv2.FONT_HERSHEY_DUPLEX, 1, COLOR_PINK, 1, lineType=cv2.LINE_AA)
                        frame = cv2.putText(frame, 'A/S/D: +5 on azim/elev/roll, Z/X/C: -5 on\"\" \"\"', (10, 120), cv2.FONT_HERSHEY_DUPLEX, 1, COLOR_PINK, 1, lineType=cv2.LINE_AA)
                        frame = cv2.putText(frame, 'F/V to lengthen/shorten the z-axis.', (10, 150), cv2.FONT_HERSHEY_DUPLEX, 1, COLOR_PINK, 1, lineType=cv2.LINE_AA)
                        frame = cv2.putText(frame, 'Press p again to resume.', (10,180), cv2.FONT_HERSHEY_DUPLEX, 1, COLOR_PINK, 1, lineType=cv2.LINE_AA)
                        cv2.imshow('image. press q to quit, p to pause, n to pass', frame)
                        key = cv2.waitKey() & 0xFF

                        if key == ord('a'):
                            angle_azim += 5
                        elif key == ord('s'):
                            angle_elev += 5
                        elif key == ord('d'):
                            angle_roll += 5
                        elif key == ord('z'):
                            angle_azim -= 5
                        elif key == ord('x'):
                            angle_elev -= 5
                        elif key == ord('c'):
                            angle_roll -= 5
                        elif key == ord('f'):
                            z_ratio += (extent['z']['M'] - extent['z']['m'])*0.02
                        elif key == ord('v'):
                            z_ratio = max(z_ratio - (extent['z']['M'] - extent['z']['m'])*0.02, 0.1)
                        elif key == ord('n'):
                            vid_next = True
                            cv2.destroyAllWindows()
                            cap.release()
                            break
                        elif key == ord('p'):
                            break
                        elif key == ord('q'):
                            cv2.destroyAllWindows()
                            cap.release()
                            return 0
                        else:
                            continue
                        
                        img_plot = draw_line_plot1(pose_xyz_dic, angle_elev, angle_azim, angle_roll, new_w, new_h, z_ratio, extent)
                        img_plot = cv2.putText(img_plot, f'azim: {angle_azim}, elev: {angle_elev}, roll: {angle_roll}', (10, 60), cv2.FONT_HERSHEY_DUPLEX, 1, COLOR_PINK, 1, lineType=cv2.LINE_AA)
                        cv2.imshow('plot', img_plot)
                    if vid_next == True: break
                frame_count += 1
                

            cap.release()
    return 0

In [107]:
a = pose_3D(video_root_BY, json_root_BY)